<a href="https://colab.research.google.com/github/amyjchao/data-course-sample/blob/main/ACHW1_Rule_based_RecSys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# 此內容會顯示為程式碼
```

# HW1 - 實作「rule-based」的推薦系統

## 基礎建設

In [1]:
import pandas as pd
import gzip, json
from random import randrange

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2021-12-26 13:47:52--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.7’

All_Beauty.csv.7    100%[===================>]  14.78M  8.26MB/s    in 1.8s    

2021-12-26 13:47:55 (8.26 MB/s) - ‘All_Beauty.csv.7’ saved [15499476/15499476]

--2021-12-26 13:47:55--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.7’

meta_All_Beauty.jso 100%[===================>]   9.85M  6.60MB/s    in 1.5s   

In [3]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [4]:
print(metadata.shape)
metadata.head()

(32892, 19)


,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [5]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [6]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

In [7]:
new_metadata = metadata[['asin', 'brand', 'title', 'price', 'also_buy', 'also_view','rank']]
new_metadata.head()

,asin,brand,title,price,also_buy,also_view,rank
0,6546546450,idea village,Loud 'N Clear&trade; Personal Sound Amplifier,,[],[],"2,938,573 in Beauty & Personal Care ("
1,7178680776,,No7 Lift &amp; Luminate Triple Action Serum 50...,$44.99,"[B01E7LCSL6, B008X5RVME]",[],"872,854 in Beauty & Personal Care ("
2,7250468162,No7,No7 Stay Perfect Foundation Cool Vanilla by No7,$28.76,[],"[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","956,696 in Beauty & Personal Care ("
3,7367905066,,Wella Koleston Perfect Hair Colour 44/44 Mediu...,,[B0041PBXX8],[],"1,870,258 in Beauty & Personal Care ("
4,7414204790,Pirmal Healthcare,Lacto Calamine Skin Balance Oil control 120 ml...,$12.15,[],"[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","67,701 in Beauty & Personal Care ("


In [8]:
new_metadata['category'] = new_metadata['rank'].str.split(' in ').str[1]
new_metadata['rank'] = new_metadata['rank'].str.split(expand = True)[0]
new_metadata['category'] = new_metadata['category'].str[:-2]
new_metadata['category'].unique()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

array(['Beauty & Personal Care', 'Beauty &amp; Personal Care', nan,
       'Grocery & Gourmet Food', 'Grocery &amp; Gourmet Food',
       'Health &amp; Household', 'Sports & Outdoors',
       'Health & Household', 'Toys & Games', 'Baby',
       'Clothing, Shoes & Jewelry', 'Home & Kitchen', 'Automotive',
       'Tools & Home Improvement'], dtype=object)

In [9]:
new_metadata['price'] = new_metadata['price'].str.replace('$','')
new_metadata['price'] = pd.to_numeric(new_metadata['price'], errors = 'coerce')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
new_metadata['rank'] = new_metadata['rank'].str.replace(',','')
new_metadata['rank'] = pd.to_numeric(new_metadata['rank'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
new_metadata['also_buy'] = new_metadata['also_buy'].astype('object')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
new_metadata.head()

,asin,brand,title,price,also_buy,also_view,rank,category
0,6546546450,idea village,Loud 'N Clear&trade; Personal Sound Amplifier,NaN,[],[],2938573.0,Beauty & Personal Care
1,7178680776,,No7 Lift &amp; Luminate Triple Action Serum 50...,44.99,"[B01E7LCSL6, B008X5RVME]",[],872854.0,Beauty & Personal Care
2,7250468162,No7,No7 Stay Perfect Foundation Cool Vanilla by No7,28.76,[],"[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]",956696.0,Beauty & Personal Care
3,7367905066,,Wella Koleston Perfect Hair Colour 44/44 Mediu...,NaN,[B0041PBXX8],[],1870258.0,Beauty & Personal Care
4,7414204790,Pirmal Healthcare,Lacto Calamine Skin Balance Oil control 120 ml...,12.15,[],"[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...",67701.0,Beauty & Personal Care


## 資料切分

In [13]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

## 產生推薦

In [14]:
import datetime

In [15]:
def random_recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    for user in users:
      temp = []
      for i in range(k):
        temp.append(metadata['asin'][randrange(32892)])
      recommendations[user] = temp

    return recommendations


#rand_ratings_by_user = random_recommender(ratings_trainings, users)
#rand_ratings_by_user

In [16]:
def recommender(training_data, users=[], k=10,wk=52):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    training_data = training_data[(training_data['DATE'] + datetime.timedelta(weeks = wk)) > '2018-09-01']
    recommendations = {}
    top_item = new_metadata.sort_values(by=['rank'])['asin'].tolist()
    top_k = top_item[:k]

    for user in users:
      user_review = training_data[training_data['reviewerID'] == user].sort_values(['DATE'], ascending=[False])
      user_review = user_review['asin'].unique()
      rec = []

      if len(user_review) > 0:
        for item in user_review:
          also_buy = new_metadata[new_metadata['asin'] == item]['also_buy'].tolist()
          also_view = new_metadata[new_metadata['asin'] == item]['also_view'].tolist()
          cate = str(new_metadata[new_metadata['asin'] == item]['category'])[9:-30]
          brand = str(new_metadata[new_metadata['asin'] == item]['brand'])[9:]
          #print(cate,brand)
          if len(also_buy) > 0:
            rec.extend(also_buy)
          
          if len(also_view) > 0:
            rec.extend(also_view)

          same_cate = new_metadata[new_metadata['category'] == cate]['asin']
          if len(brand) > 0:
            same_brand = new_metadata[new_metadata['brand'] == brand]['asin']
            same_brand_cate = set(same_cate) & set(same_brand)
            
            rec.extend(list(same_brand_cate))
            rec.extend(same_brand.tolist())      
          
          #same_cate = new_metadata[:k][new_metadata['cate'] == cate]
          rec.extend(same_cate.tolist())
      
      #top k item in whole store for the rest

      while [] in rec:
        rec.remove([])

      rec = [item for sublist in rec for item in sublist]
      rec.extend(top_k)
      rec = rec[:k]
      recommendations[user] = rec

    return recommendations


#ratings_by_user = recommender(ratings_trainings, users,10, 12)
#ratings_by_user

In [29]:
def top_recommender(training_data, users=[], k=10, rating=3):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    product_rank = new_metadata[['asin','rank']]
    combine = ratings_trainings.set_index('asin').join(product_rank.set_index('asin'))
    mean_df = combine.groupby('asin').mean().reset_index()
    df = mean_df[mean_df.overall >= rating].sort_values('rank').head(k) 

    recommendations = {user: df['asin'] for user in users}

    return recommendations


#ratings_by_user = recommender(ratings_trainings, users,10, 12)
#ratings_by_user

## 結果評估

In [18]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

#evaluate(ratings_testings_by_user, rand_ratings_by_user)
#evaluate(ratings_testings_by_user, ratings_by_user)

## 數據比較

In [19]:
import numpy as np

In [20]:
ks = range(10,51,10)
wks = [1, 2, 4, 8, 12, 24, 52, 104]
scores = []
for k in ks:
    for wk in wks:    
        ratings_by_user = recommender(ratings_trainings, users, k=k, wk=wk)
        score = evaluate(ratings_testings_by_user, ratings_by_user)
        scores.append(score)
        print("k = %d, weeks = %d, score = %.5f" %(k, wk, score))

pd.DataFrame(np.reshape(scores, (5, 8)).T, index=pd.Index(wks, name='Weeks'), columns=pd.Index(ks, name='k'))

k = 10, weeks = 1, score = 0.00508
k = 10, weeks = 2, score = 0.00508
k = 10, weeks = 4, score = 0.00508
k = 10, weeks = 8, score = 0.00508
k = 10, weeks = 12, score = 0.00508
k = 10, weeks = 24, score = 0.00508
k = 10, weeks = 52, score = 0.00508
k = 10, weeks = 104, score = 0.00508
k = 20, weeks = 1, score = 0.01186
k = 20, weeks = 2, score = 0.01186
k = 20, weeks = 4, score = 0.01186
k = 20, weeks = 8, score = 0.01356
k = 20, weeks = 12, score = 0.01356
k = 20, weeks = 24, score = 0.01356
k = 20, weeks = 52, score = 0.01356
k = 20, weeks = 104, score = 0.01356
k = 30, weeks = 1, score = 0.02034
k = 30, weeks = 2, score = 0.02034
k = 30, weeks = 4, score = 0.02034
k = 30, weeks = 8, score = 0.02203
k = 30, weeks = 12, score = 0.02203
k = 30, weeks = 24, score = 0.02203
k = 30, weeks = 52, score = 0.02203
k = 30, weeks = 104, score = 0.02203
k = 40, weeks = 1, score = 0.03051
k = 40, weeks = 2, score = 0.03051
k = 40, weeks = 4, score = 0.03051
k = 40, weeks = 8, score = 0.03220
k = 4

k,10,20,30,40,50
Weeks,,,,,
1,0.005085,0.011864,0.020339,0.030508,0.042373
2,0.005085,0.011864,0.020339,0.030508,0.042373
4,0.005085,0.011864,0.020339,0.030508,0.042373
8,0.005085,0.013559,0.022034,0.032203,0.044068
12,0.005085,0.013559,0.022034,0.032203,0.044068
24,0.005085,0.013559,0.022034,0.032203,0.044068
52,0.005085,0.013559,0.022034,0.032203,0.044068
104,0.005085,0.013559,0.022034,0.030508,0.042373


In [32]:
ks = range(10,51,10)
ratings = [3, 3.5, 4, 4.5]
scores = []
for k in ks:
    for rate in ratings:    
        ratings_by_user = top_recommender(ratings_trainings, users, k=k, rating=rate)
        score = evaluate(ratings_testings_by_user, ratings_by_user)
        scores.append(score)
        print("k = %d, rating = %.5f, score = %.5f" %(k, rate, score))

pd.DataFrame(np.reshape(scores, (5, 4)).T, index=pd.Index(ratings, name='Ratings'), columns=pd.Index(ks, name='k'))

k = 10, rating = 3.00000, score = 0.00508
k = 10, rating = 3.50000, score = 0.00508
k = 10, rating = 4.00000, score = 0.01017
k = 10, rating = 4.50000, score = 0.01017
k = 20, rating = 3.00000, score = 0.01186
k = 20, rating = 3.50000, score = 0.01186
k = 20, rating = 4.00000, score = 0.02034
k = 20, rating = 4.50000, score = 0.02203
k = 30, rating = 3.00000, score = 0.02034
k = 30, rating = 3.50000, score = 0.02034
k = 30, rating = 4.00000, score = 0.03051
k = 30, rating = 4.50000, score = 0.02712
k = 40, rating = 3.00000, score = 0.03559
k = 40, rating = 3.50000, score = 0.03559
k = 40, rating = 4.00000, score = 0.04237
k = 40, rating = 4.50000, score = 0.02712
k = 50, rating = 3.00000, score = 0.04237
k = 50, rating = 3.50000, score = 0.04237
k = 50, rating = 4.00000, score = 0.05254
k = 50, rating = 4.50000, score = 0.03220


k,10,20,30,40,50
Ratings,,,,,
3.0,0.005085,0.011864,0.020339,0.035593,0.042373
3.5,0.005085,0.011864,0.020339,0.035593,0.042373
4.0,0.010169,0.020339,0.030508,0.042373,0.052542
4.5,0.010169,0.022034,0.027119,0.027119,0.032203


## 測試邏輯

In [ ]:
user = 'A1210QJT54O8T0'
user_bought = ratings[ratings['reviewerID'] == user]['asin'].unique()
rec = []

for item in user_bought:
  cate = str(new_metadata[new_metadata['asin'] == item]['category'])
  brand = str(new_metadata[new_metadata['asin'] == item]['brand'])
  same_cate = new_metadata[new_metadata['category'] == cate]
  print(cate[9:-30])
'''
  #print(len(new_metadata[new_metadata['asin'] == item]['also_buy'].tolist()))

  if len(new_metadata[new_metadata['asin'] == item]['also_buy'].tolist()) != 0:
    rec.extend(new_metadata[new_metadata['asin'] == item]['also_buy'].tolist())

  if len(new_metadata[new_metadata['asin'] == item]['also_view'].tolist()) != 0:
    rec.extend(new_metadata[new_metadata['asin'] == item]['also_view'].tolist())
  

  if len(brand) > 0:
    same_brand_cate = new_metadata[new_metadata['brand'] == brand]  & new_metadata[new_metadata['category'] == cate] 
  same_brand_cate = new_metadata[new_metadata['category']== cate]

  
while [] in rec:
  rec.remove([])
flat_list = [item for sublist in rec for item in sublist]
print(flat_list)
'''